# Chain of thoughts

In this notebook we will try to implement "chain of thoughts" state-machine.
It will use LLM to generate thoughts, actions and answers and will use tools to get results.

We have only 3 dependencies:
- transitions - to implement state-machine
- transformers - to use LLM
- accelerate - to use mixed precision for LLM

In [1]:
%pip install -q transitions transformers accelerate

Note: you may need to restart the kernel to use updated packages.


## Create instance of LLM

We will use T5 model from HuggingFace. But it could be any other LLM of relevant or greater size.

In [2]:
from transformers import pipeline
import torch

model_id = 'google/flan-t5-xxl'
default_pipeline = pipeline(
    task='text2text-generation',
    model=model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

## Implementation

We will create a class which implements "chain of thoughts" state-machine.
It will use LLM to generate thoughts, actions and answers and will use tools to get results.

Current state-machine will have following states:
- INPUT - initial state, we will ask user a question
- THOUGHT - we will use LLM to generate THOUGHT (answer or action)
- ACTION - we will use LLM to generate ACTION (tool name)
- ACTION_INPUT - we will ask user to provide input for ACTION
- OBSERVATION - we will use LLM to generate OBSERVATION (result of tool usage)
- ANSWER - we will use LLM to generate ANSWER (final answer)

Transitions are running in following order:
- INPUT -> THOUGHT -> ACTION -> ACTION_INPUT -> OBSERVATION -> THOUGHT -> ... -> THOUGHT -> ANSWER

In [3]:
import logging
from enum import Enum

from transitions import Machine

torch.cuda.empty_cache()

THOUGHT_ELEMENTS = Enum('THOUGHT_ELEMENTS', ['INPUT', 'THOUGHT', 'ACTION', 'ACTION_INPUT', 'OBSERVATION', 'ANSWER'])

logging.basicConfig(level=logging.INFO)


class ChainOfThoughts(Machine):
    """
        Following class implements chain-of-thoughts state-machine.
        LLM will decide when the answer for prompt is ready as well when it needs to use particular TOOL.
    """
    states = THOUGHT_ELEMENTS
    transitions = [
        {'trigger': 'ask', 'source': THOUGHT_ELEMENTS.INPUT, 'dest': THOUGHT_ELEMENTS.THOUGHT},
        {'trigger': 'act', 'source': THOUGHT_ELEMENTS.THOUGHT, 'dest': THOUGHT_ELEMENTS.ACTION},
        {'trigger': 'answer', 'source': THOUGHT_ELEMENTS.THOUGHT, 'dest': THOUGHT_ELEMENTS.ANSWER},
        {'trigger': 'pass_params', 'source': THOUGHT_ELEMENTS.ACTION, 'dest': THOUGHT_ELEMENTS.ACTION_INPUT},
        {'trigger': 'observe', 'source': THOUGHT_ELEMENTS.ACTION_INPUT, 'dest': THOUGHT_ELEMENTS.OBSERVATION},
        {'trigger': 'think', 'source': THOUGHT_ELEMENTS.OBSERVATION, 'dest': THOUGHT_ELEMENTS.THOUGHT},
    ]

    tools = None
    llm_pipeline = None

    def __init__(self,
                 name: str,
                 tools: list,
                 llm_pipeline: pipeline
                 ):
        super().__init__(
            states=ChainOfThoughts.states,
            transitions=ChainOfThoughts.transitions,
            initial=THOUGHT_ELEMENTS.INPUT,
            send_event=True,
        )
        self.name = name
        self.tools = tools
        self.llm_pipeline = llm_pipeline

    def on_enter_THOUGHT(self, events):
        """
            This method will be called when we enter THOUGHT state.
            We will use LLM to generate THOUGHT.
            It will be one of the tools from the list.
            If llm can produce answer without tool usages - we will go to ANSWER state.
            If llm has to use tool and this tool exists in the list - we will go to ACTION state.
        """
        question = events.kwargs.get('question')
        history = events.kwargs.get('history', [])

        tools_list = ", ".join( [f"{tool['tool_name']} - {tool['description']}" for tool in self.tools])

        if len(history) == 0:
            history.append("System: AI is here to help you.")
            history.append("User: " + question)

        history.append("System: AI has following tools to help you:" + tools_list)
        history.append("System: Do you need to use any of these tools to answer(yes/no)?")
        prompt = '\nAI: Between yes and no, i will choose: '

        assembled_prompt = "\n".join(history) + prompt
        tool_required = self.llm_pipeline(assembled_prompt)[0]['generated_text']
        if 'yes' in tool_required.lower():
            history.append("AI: I need to use a tool.")
            self.act(
                history=history,
                question=question
            )
        elif 'no' in tool_required.lower():
            history.append("AI: I can answer without a tool.")
            self.answer(
                history=history,
                question=question
            )
        else:
            logging.error(f'{self.name} is confused. Cannot understand tools requirements, got {tool_required}')
            logging.log(f'History: {history}')

    def on_enter_ACTION(self, events):
        """
            This method will be called when we enter ACTION state.
            We will use LLM to generate ACTION.
            It will be one of the tools from the list.
            If llm can produce answer without tool usages - we will go to ANSWER state.
            If llm has to use tool and this tool exists in the list - we will go to ACTION state.
        """
        history = events.kwargs.get('history', [])
        question = events.kwargs.get('question')
        history.append("System: Write a name of the tool you want to use")
        prompt = f"AI: I will use a tool named: "
        assembled_prompt = "\n".join(history) + prompt
        action = self.llm_pipeline(assembled_prompt)[0]['generated_text']
        if action in [tool['tool_name'] for tool in self.tools]:
            history.append(f"AI: {action}")
            self.pass_params(
                history=history,
                action=action,
                question=question
            )
        else:
            logging.error(f'{self.name} is confused. Cannot understand tool name {action}')
            logging.debug(f'History: {history}')

    def on_enter_ACTION_INPUT(self, events):
        """
            This method will be called when we enter ACTION_INPUT state.
            We will use tool to get result.
            After that we will go to THOUGHT state again.
        """
        history = events.kwargs.get('history', [])
        question = events.kwargs.get('question')
        action = events.kwargs.get('action')
        prompt = "AI: I need to pass there following input - "
        assembled_prompt = "\n".join(history) + prompt
        action_input = self.llm_pipeline(assembled_prompt)[0]['generated_text']
        history.append(f"AI: {action_input}")
        self.observe(
            history=history,
            action=action,
            action_input=action_input,
            question=question
        )

    def on_enter_OBSERVATION(self, events):
        """
            This method will be called when we enter OBSERVATION state.
            We will use tool to get result.
            After that we will go to THOUGHT state again.
        """
        history = events.kwargs.get('history', [])
        question = events.kwargs.get('question')
        action = events.kwargs.get('action')
        action_input = events.kwargs.get('action_input')

        tool = [tool for tool in self.tools if tool['tool_name'] == action][0]['func']
        result = tool(action_input)
        history.append(f"System: This tool produced next result: {result}")
        self.think(history=history, question=question)

    def on_enter_ANSWER(self, events):
        """
            This method will be called when we enter ANSWER state.
            We will use LLM to generate answer.
        """
        history = events.kwargs.get('history', [])
        question = events.kwargs.get('question')
        prompt = f"AI: Your original question was - {question}. And during my research I found following answer - "
        assembled_prompt = "\n".join(history) + prompt
        answer = self.llm_pipeline(assembled_prompt)[0]['generated_text']
        history.append(f"{prompt}{answer}")
        history.append("System: Thank you for using AI. Goodbye!")
        print("\n".join(history))


## Usage

It the current example we provided two stubs for tools - one is for getting articles from Wikipedia and another one is for googling.

In [4]:
tools = [
    {
        "tool_name": "Google",
        "description": "useful when you need to find some actual information",
        "func": lambda query: "Huge amount of people are protesting in Brno today is a news item",
    },
    {
        "tool_name": "Wikipedia",
        "description": "useful when you need to get encyclopedic information about something",
        "func": lambda query: "404: Page not found",
    }
]

solver = ChainOfThoughts(name='solver', llm_pipeline=default_pipeline, tools=tools)

solver.ask(question='What is the recent news in Czechia?')

INFO:transitions.core:solverFinished processing state INPUT exit callbacks.
INFO:transitions.core:solverFinished processing state THOUGHT exit callbacks.
INFO:transitions.core:solverFinished processing state ACTION exit callbacks.
INFO:transitions.core:solverFinished processing state ACTION_INPUT exit callbacks.
INFO:transitions.core:solverFinished processing state OBSERVATION exit callbacks.
INFO:transitions.core:solverFinished processing state THOUGHT exit callbacks.
INFO:transitions.core:solverExecuted callback 'on_enter_ANSWER'
INFO:transitions.core:solverFinished processing state ANSWER enter callbacks.
INFO:transitions.core:solverExecuted callback 'on_enter_THOUGHT'
INFO:transitions.core:solverFinished processing state THOUGHT enter callbacks.
INFO:transitions.core:solverExecuted callback 'on_enter_OBSERVATION'
INFO:transitions.core:solverFinished processing state OBSERVATION enter callbacks.
INFO:transitions.core:solverExecuted callback 'on_enter_ACTION_INPUT'
INFO:transitions.c

System: AI is here to help you.
User: What is the recent news in Czechia?
System: AI has following tools to help you:Google - useful when you need to find some actual information, Wikipedia - useful when you need to get encyclopedic information about something
System: Do you need to use any of these tools to answer(yes/no)?
AI: I need to use a tool.
System: Write a name of the tool you want to use
AI: Google
AI: Search for "Czech Republic news"
System: This tool produced next result: Huge amount of people are protesting in Brno today is a news item
System: AI has following tools to help you:Google - useful when you need to find some actual information, Wikipedia - useful when you need to get encyclopedic information about something
System: Do you need to use any of these tools to answer(yes/no)?
AI: I can answer without a tool.
AI: Your original question was - What is the recent news in Czechia?. And during my research I found following answer - Huge amount of people are protesting in 

True